In [5]:
import pandas as pd
import json
import os
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
import openai
from langchain.llms import OpenAI
from langchain.chains import GraphCypherQAChain
from langchain_community.llms import HuggingFaceHub
load_dotenv()

True

In [6]:
# Hugging Face (if we want to use open source LLM)
hf_api = os.getenv("HF_API")

neo4j_url = "neo4j+ssc://63de7d0e.databases.neo4j.io"
neo4j_user = "neo4j"
neo4j_password = "bkR1Ib1eerfMqlKdG7B2wOWuS5ki8qiJ7kivxx_5zKM"

# Kết nối với Neo4j
graph = Neo4jGraph(neo4j_url, neo4j_user, neo4j_password)


C:\Users\haian\AppData\Local\Temp\ipykernel_15876\287066134.py:9: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(neo4j_url, neo4j_user, neo4j_password)


Thanks to Manish Kumar, for providing this dataset in Kaggle (https://www.kaggle.com/datasets/manishkumar7432698/linkedinuserprofiles?select=LinkedIn+people+profiles+datasets.csv)!

# 3. Query through our Knowledge Graph

In [7]:
openai_api_key = os.getenv("CHATGPT_API")


# 4. Prompting Strategies

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain

# Use ChatOpenAI with the correct model (gpt-3.5-turbo)
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=openai_api_key, temperature=0)

In [57]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from language_cypher import examples_language
from automation_tool import automation_tool_cypher

all_examples = examples_language + automation_tool_cypher


example_prompt = PromptTemplate(
    input_variables=["question", "query"],
    template="User input: {question}\nCypher query: {query}"
)

prompt = FewShotPromptTemplate(
    examples=all_examples,
    example_prompt=example_prompt,
    prefix=(
        "You are a Neo4j expert. Given an input question, create a syntactically "
        "correct Cypher query to run.\n\nHere is the \n{schema} information:\n\n"
        "Below are a number of examples of questions and their corresponding Cypher queries.\n"
        "If the query returns results, answer the question based on those results.\n\n"
    ),

    suffix="User input: {question}\nCypher query: ",
    input_variables=["question", "schema"],  # Ensure schema is passed correctly when calling prompt

)


In [58]:
chain = GraphCypherQAChain.from_llm(
    graph=graph,
    llm=llm,
    cypher_prompt=prompt,
    verbose=True,
    allow_dangerous_requests=True,
)

In [59]:
response = chain.invoke(q)
print(response)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (e:Employee)-[:HAS_LANGUAGE_LEVEL]->(le:Level) WHERE le.name IN ['IELTS 6.0', 'IELTS 6.5', 'IELTS 7.0'] Return e.name
Full Context:
[{'e.name': 'Nguyen Cong Tuan P'}]

> Finished chain.
{'query': 'Find employees have IELTS scores between 6.0 and 7.0', 'result': 'Nguyen Cong Tuan P has IELTS scores between 6.0 and 7.0.'}


In [60]:
questions = ["Find employees have IELTS scores between 6.0 and 7.0",
             "Find employees using Terraform"]
for q in questions:
    print('====== START ======')
    print(chain.invoke(q)['result'])
    print('====== END ====== \n')

====== START ======


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (e:Employee)-[:HAS_LANGUAGE_LEVEL]->(le:Level) WHERE le.name IN ['IELTS 6.0', 'IELTS 6.5', 'IELTS 7.0'] Return e.name
Full Context:
[{'e.name': 'Nguyen Cong Tuan P'}]

> Finished chain.
Nguyen Cong Tuan P has IELTS scores between 6.0 and 7.0.
====== END ====== 

====== START ======


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (e:Employee)-[:AUTOTOOL]->(automationTool:AutomationTool {name: 'Terraform'}) RETURN e.name
Full Context:
[{'e.name': 'Bui Duc A'}, {'e.name': 'Pham Viet H'}, {'e.name': 'Nguyen Quang H'}, {'e.name': 'Huynh Thanh T'}, {'e.name': 'Luong Ba H'}, {'e.name': 'Hoang Dang K'}, {'e.name': 'Phuong Xuan T'}]

> Finished chain.
I don't know the answer.
====== END ====== 



In [171]:
print(prompt.format(question="Where do Michael work?", schema="foo"))

You are a Neo4j expert. Given an input question, create a syntactically correct Cypher query to run.

Here is the schema information
foo.

Below are a number of examples of questions and their corresponding Cypher queries.

User input: Which workers speak French?
Cypher query: MATCH (p:Person)-[:SPEAKS]->(l:Language {name: 'French'}) RETURN p.name

User input: What industries are workers named Emily associated with?
Cypher query: MATCH (p:Person {name: 'Emily'})-[:WORKS_IN]->(c:Company)-[:IS_IN]->(i:Industry) RETURN i.name

User input: Which workers live in Canada and speak German?
Cypher query: MATCH (p:Person)-[:LIVES_IN]->(:Country {name: 'Canada'}), (p)-[:SPEAKS]->(:Language {name: 'German'}) RETURN p.name

User input: Where do Michael work?
Cypher query: 


In [ ]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate.from_template(
    "User input: {question}\nCypher query: {query}"
)
prompt = FewShotPromptTemplate(
    examples=examples[:3],
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. Given an input question, create a syntactically correct Cypher query to run.\n\nHere is the schema information\n{schema}.\n\nBelow are a number of examples of questions and their corresponding Cypher queries.",
    suffix="User input: {question}\nCypher query: ",
    input_variables=["question", "schema"],
)

In [ ]:
examples= [
    {
        "question": "Which workers speak French?",
        "query": "MATCH (p:Person)-[:SPEAKS]->(l:Language {{name: 'French'}}) RETURN p.name",
    },
    {
        "question": "What industries are workers named Emily associated with?",
        "query": "MATCH (p:Person {{name: 'Emily'}})-[:WORKS_IN]->(c:Company)-[:IS_IN]->(i:Industry) RETURN i.name",
    },
    {
        "question": "Which workers live in Canada and speak German?",
        "query": "MATCH (p:Person)-[:LIVES_IN]->(:Country {{name: 'Canada'}}), (p)-[:SPEAKS]->(:Language {{name: 'German'}}) RETURN p.name",
    },
    {
        "question": "In which countries do workers who speak Spanish live?",
        "query": "MATCH (p:Person)-[:SPEAKS]->(:Language {{name: 'Spanish'}})<-[:SPEAKS]-(worker:Person)-[:LIVES_IN]->(c:Country) RETURN DISTINCT c.name AS Country",
    },
    {
        "question": "What companies do workers named John work in?",
        "query": "MATCH (p:Person {{name: 'John'}})-[:WORKS_IN]->(c:Company) RETURN c.name",
    },
    {
        "question":"How many workers in Hospital and Health Care industry able to speak Korea",
        "query": "MATCH (p:Person)-[:WORKS_IN]->(:Company)-[:IS_IN]->(:Industry {{name: 'Hospitals and Health Care'}}),(p)-[:SPEAKS]->(:Language {{name: 'Korean'}}) RETURN COUNT(DISTINCT p) AS NumberOfWorkers",
    },
    {
        "question": "What companies are located in the technology industry?",
        "query": "MATCH (c:Company)-[:IS_IN]->(:Industry {{name: 'Technology'}}) RETURN c.name",
    },
    {
        "question": "Where do workers named Alice live?",
        "query": "MATCH (p:Person {{name: 'Alice'}})-[:LIVES_IN]->(c:Country) RETURN c.name",
    },
]


In [148]:
from langchain_community.vectorstores import Neo4jVector
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.embeddings import HuggingFaceEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    HuggingFaceEmbeddings(),
    Neo4jVector,
    url = neo4j_url,
    username = neo4j_user,
    password = neo4j_password,
    k=3,
    input_keys=["question"],
)

C:\Users\haian\AppData\Local\Temp\ipykernel_2504\2708866189.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  HuggingFaceEmbeddings(),
C:\Users\haian\AppData\Local\Temp\ipykernel_2504\2708866189.py:7: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  HuggingFaceEmbeddings(),
C:\Users\haian\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/st

KeyboardInterrupt: 

In [ ]:
example_selector.select_examples({"question": "Where do Michael work?"})

In [21]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector, #previous: examples = examples[:3]
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. Given an input question, create a syntactically correct Cypher query to run.\n\nHere is the schema information\n{schema}.\n\nBelow are a number of examples of questions and their corresponding Cypher queries.",
    suffix="User input: {question}\nCypher query: ",
    input_variables=["question", "schema"],
)

In [27]:
print(dynamic_prompt.format(question="Where do Michael work?", schema="foo"))

You are a Neo4j expert. Given an input question, create a syntactically correct Cypher query to run.

Here is the schema information
foo.

Below are a number of examples of questions and their corresponding Cypher queries.

User input: What companies do workers named John work in?
Cypher query: MATCH (p:Person {name: 'John'})-[:WORKS_IN]->(c:Company) RETURN c.name

User input: Where do workers named Alice live?
Cypher query: MATCH (p:Person {name: 'Alice'})-[:LIVES_IN]->(c:Country) RETURN c.name

User input: What industries are workers named Emily associated with?
Cypher query: MATCH (p:Person {name: 'Emily'})-[:WORKS_IN]->(c:Company)-[:IS_IN]->(i:Industry) RETURN i.name

User input: Where do Michael work?
Cypher query: 
